In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from Scripts.video import get_frame_information

In [2]:
# global directory path variables. make these your folder names under MCS
ICATCHER_DIR = 'iCatcherOutput'

# trial info
TRIAL_INFO_DIR = 'lookit_info/lookit_trial_timing_info.csv'

# directory for videos
VID_DIR = '/nese/mit/group/saxelab/users/galraz/mcs/videos/BBB'

# add absolute path to iCatcher repo
ICATCHER = '/Users/gracesong/dev/iCatcher'
sys.path.append(ICATCHER)

In [3]:
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f


In [4]:
def run_analyze_output(data_filename="BBB_output.csv", session=None):
    """
    Given an iCatcher output directory and Datavyu input and output 
    files, runs iCatcher over all videos in vid_dir that have not been
    already run, computes looking times for all iCatcher outputs, and
    compares with Datavyu looking times. 
    data_filename (string): name of file you want comparison data to be written
            to. Must have .csv ending. 
    session (string): ID of the experiment session. If session is not
            specified, looks for videos only within VID_DIR, otherwise
            searches within [VID_DIR]/session[session]
    """
    for filename in listdir_nohidden(ICATCHER_DIR):
        child_id = filename.split('.')[0]

        # skip if child data already added
        output_file = Path(data_filename)
        if output_file.is_file():
            output_df = pd.read_csv(data_filename, index_col=0)
            ids = output_df['child'].unique()
            if child_id in ids: 
                print(child_id + ' already processed')
                continue
        
        vid_path = VID_DIR + '/'
        if session:
            vid_path += "session" + session + '/'
        vid_path = vid_path + child_id + ".mp4"

        # get timestamp for each frame in the video
        print('getting frame information for {}...'.format(vid_path))
        timestamps, length = get_frame_information(vid_path)
        if not timestamps:
            print('video not found for {} in {} folder'.format(child_id, VID_DIR))
            continue
        
        # initialize df with time stamps for iCatcher file
        icatcher_path = ICATCHER_DIR + '/' + filename
        icatcher = read_convert_output(icatcher_path, timestamps)

        # get trial onsets and offsets from input file, match to iCatcher file
        trial_sets, df = get_trial_sets(child_id)
        assign_trial(icatcher, trial_sets)
        
        # sum on looks and off looks for each trial
        icatcher_times = get_on_off_times(icatcher)
        # datavyu_times = get_output_times(output_file)

        # check whether number of trials from trial info is the same as 
        if icatcher['trial'].max() != len(df):
            print('mismatch in # of trials between icatcher and session info: {} in {} folder'.format(child_id, VID_DIR))
            continue

        write_to_csv(data_filename, child_id, icatcher_times, session, df['fam_or_test'], df['scene'], icatcher)
